In [2]:
!pip install datasets bitsandbytes trl==0.12.1 transformers peft huggingface-hub accelerate safetensors pandas matplotlib numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 126.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 39.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requ

# Install necessary libraries

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    #AutoPeftModelForCausalLM, # Removed from transformers
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer, SFTConfig
# from trl.trainer.utils import DataCollatorForCompletionOnlyLM
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM, PeftConfig # Added to peft
from huggingface_hub import notebook_login
from trl import SFTTrainer, SFTConfig, setup_chat_format, DataCollatorForCompletionOnlyLM


# Check for bf16 support and set compute dtype


In [2]:
support = torch.cuda.is_bf16_supported(including_emulation=False)
calculate_dtype = torch.bfloat16 if support else torch.float32

In [3]:
print(calculate_dtype)

torch.bfloat16


#bnb config for loading 4 bit model with nf4 quant type
* loading model with quantization config
* device map to cuda
* 4bit true

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= calculate_dtype, #calculate_dtype can be bf16 or float32- use bf16 if supported
    bnb_4bit_use_double_quant= True
    )
repo = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(repo, quantization_config= bnb_config, device_map= "cuda:0")

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

#Check model memory footprint

In [5]:
print(model.get_memory_footprint()/1024/1024)

2104.1310424804688


#model config

In [6]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLUActivation()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=

#Prepare model for kbit training
##Use Lora Config


1.   rank [4,8,16,32] - choose one
2.   lora_alpha is a scalling factor which should be 2x the rank of matrix.
3.   dropout range from 0.03 to 0.10 which helps prevent overfit
4.   module - choose module as per requirement


In [7]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r = 8, #. rank of LoRA - [4-16]
    bias = "none", # ["all", "lora_only"] - for train bias term
    lora_alpha = 16, # scalling factor
    lora_dropout = 0.10, # prevent overfit- used for regularisation
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type = "CAUSAL_LM"

)

model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (q

#once again check memory footprint

In [8]:
print(model.get_memory_footprint()/1024/1024)

2497.2619018554688


#Print base model to compare

In [9]:
print(model.get_base_model)

<bound method PeftModel.get_base_model of PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): Mod

In [10]:
print(model.get_memory_footprint()/1e6)

2618.568896


#Check for trainable Parameters and its percentage for a mathematical view.

In [11]:
trainable_params, total_params = model.get_nb_trainable_parameters()
percentage = (trainable_params / total_params) * 100

print(f"Trainable Parameters: {trainable_params:,}")
print(f"Total Parameters: {total_params:,}")
print(f"Percentage Trainable: {percentage:.2f}%")

Trainable Parameters: 4,456,448
Total Parameters: 3,825,536,000
Percentage Trainable: 0.12%


#ETL Process for Dataset Prep stage, Tokenizer load and define chat template if needed.

In [12]:
# ============================================
# SETUP: Phi-3-Mini Model & Tokenizer
# ============================================
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Add padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ============================================
# LOAD FINANCIAL Q&A DATASET
# ============================================
print("💼 Loading FinGPT Financial Q&A dataset...")

# Load financial Q&A dataset from FinGPT
raw_dataset = load_dataset(
    "FinGPT/fingpt-fiqa_qa",
    split="train[:5000]"  # Using 5k samples for comprehensive training
)

print(f"Dataset loaded: {len(raw_dataset)} samples")
print(f"Dataset columns: {raw_dataset.column_names}")
print(f"\n📋 Sample entry:")
print(raw_dataset[0])

# ============================================
# FORMAT PROMPTS FOR PHI-3
# ============================================

def format_prompt(example):
    """
    Format financial Q&A into Phi-3's instruction format

    Phi-3 chat format:
    <|system|>
    {system_message}<|end|>
    <|user|>
    {user_message}<|end|>
    <|assistant|>
    {assistant_response}<|end|>
    """

    # Extract question and answer from dataset
    # The dataset has 'instruction' (question) and 'output' (answer)
    # Some entries might also have 'input' for additional context
    question = example.get("instruction", example.get("question", ""))
    context = example.get("input", "")
    answer = example.get("output", example.get("answer", ""))

    # System message for financial expertise
    system_message = (
        "You are FinSight, an expert financial advisor and analyst. "
        "Provide accurate, detailed answers to financial questions. "
        "Use clear explanations and cite relevant financial concepts when applicable."
    )

    # Construct user message with context if available
    if context and context.strip():
        user_message = f"{question}\n\nContext: {context}"
    else:
        user_message = question

    # Format in Phi-3 chat template
    prompt = (
        f"<|system|>\n{system_message}<|end|>\n"
        f"<|user|>\n{user_message}<|end|>\n"
        f"<|assistant|>\n{answer}<|end|>"
    )

    return {"text": prompt}

# ============================================
# TOKENIZATION FUNCTION
# ============================================

def tokenize_function(examples):
    """Tokenize the formatted text with proper padding and labels"""

    # Tokenize the text
    # Using 1024 max length for detailed financial Q&A
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=1024,  # Increased for comprehensive financial answers
        return_tensors=None
    )

    # Create labels for causal LM training
    # Labels = input_ids, but -100 for padded tokens (ignored in loss)
    labels = []
    for input_ids, attention_mask in zip(tokenized["input_ids"], tokenized["attention_mask"]):
        # Convert to list if needed
        label = input_ids.copy() if isinstance(input_ids, list) else list(input_ids)

        # Set padded positions to -100 (ignored in loss calculation)
        for i, mask in enumerate(attention_mask):
            if mask == 0:  # This is a padded token
                label[i] = -100

        labels.append(label)

    tokenized["labels"] = labels
    return tokenized

# ============================================
# PROCESS DATASET
# ============================================

print("\n💬 Formatting financial Q&A prompts...")
formatted_dataset = raw_dataset.map(
    format_prompt,
    desc="Formatting financial Q&A with Phi-3 instruction template"
)

print("\n🔢 Tokenizing dataset...")
tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=formatted_dataset.column_names,  # Remove original columns
    desc="Tokenizing financial Q&A",
    num_proc=4  # Use multiple processes for faster tokenization
)

# Final dataset ready for training
final_dataset = tokenized_dataset

# ============================================
# SUMMARY & STATISTICS
# ============================================

print("\n" + "="*70)
print("✅ FINANCIAL Q&A DATASET PREPARATION COMPLETE!")
print("="*70)
print(f"💼 Total samples: {len(final_dataset)}")
print(f"📏 Max sequence length: {len(final_dataset[0]['input_ids'])}")
print(f"🔑 Dataset keys: {final_dataset.column_names}")
print(f"💾 Approximate size: {len(final_dataset) * 1024 * 4 / 1024 / 1024:.2f} MB")
print("\n🎯 Task: Financial Question Answering")
print("💡 Domain: Investment, Banking, Corporate Finance, Personal Finance")
print("🤖 Model: Phi-3-Mini-4K-Instruct (3.8B)")
print("🚀 Ready to use 'final_dataset' in your SFTTrainer!")
print("="*70)

# Optional: Print a sample to verify formatting
print("\n📋 Sample formatted prompt (first 600 chars):")
sample_text = formatted_dataset[0]["text"]
print(sample_text[:600] + "...\n")

# Show statistics about sequence lengths
print("📊 Sequence Length Statistics:")
lengths = [len([x for x in seq if x != tokenizer.pad_token_id])
           for seq in tokenized_dataset["input_ids"][:100]]
print(f"   Average length (first 100): {sum(lengths)/len(lengths):.0f} tokens")
print(f"   Min length: {min(lengths)} tokens")
print(f"   Max length: {max(lengths)} tokens")
print(f"\n💡 Tip: Longer sequences = more detailed financial answers!")

# Sample some questions to show diversity
print("\n📚 Sample Questions from Dataset:")
for i in range(3):
    q = raw_dataset[i].get("instruction", raw_dataset[i].get("question", ""))
    print(f"\n   {i+1}. {q[:150]}...")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

Dataset loaded: 5000 samples
Dataset columns: ['input', 'output', 'instruction']

📋 Sample entry:
{'input': 'What is considered a business expense on a business trip?', 'output': 'The IRS Guidance pertaining to the subject.  In general the best I can say is your business expense may be deductible.  But it depends on the circumstances and what it is you want to deduct. Travel Taxpayers who travel away from home on business may deduct related   expenses, including the cost of reaching their destination, the cost   of lodging and meals and other ordinary and necessary expenses.   Taxpayers are considered “traveling away from home” if their duties   require them to be away from home substantially longer than an   ordinary day’s work and they need to sleep or rest to meet the demands   of their work. The actual cost of meals and incidental expenses may be   deducted or the taxpayer may use a standard meal allowance and reduced   record keeping requirements. Regardless of the method used, me

Formatting financial Q&A with Phi-3 instruction template:   0%|          | 0/5000 [00:00<?, ? examples/s]


🔢 Tokenizing dataset...


Tokenizing financial Q&A (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]


✅ FINANCIAL Q&A DATASET PREPARATION COMPLETE!
💼 Total samples: 5000
📏 Max sequence length: 1024
🔑 Dataset keys: ['input_ids', 'attention_mask', 'labels']
💾 Approximate size: 19.53 MB

🎯 Task: Financial Question Answering
💡 Domain: Investment, Banking, Corporate Finance, Personal Finance
🤖 Model: Phi-3-Mini-4K-Instruct (3.8B)
🚀 Ready to use 'final_dataset' in your SFTTrainer!

📋 Sample formatted prompt (first 600 chars):
<|system|>
You are FinSight, an expert financial advisor and analyst. Provide accurate, detailed answers to financial questions. Use clear explanations and cite relevant financial concepts when applicable.<|end|>
<|user|>
Utilize your financial knowledge, give your answer or opinion to the input question or subject . Answer format is not limited.

Context: What is considered a business expense on a business trip?<|end|>
<|assistant|>
The IRS Guidance pertaining to the subject.  In general the best I can say is your business expense may be deductible.  But it depends on

In [14]:
from trl import SFTConfig, SFTTrainer

# My SFT Trainer Configuration for Phi-3-Mini Financial Q&A Fine-tuning
# Using LoRA: rank=8, alpha=16, dropout=0.1, 4-bit quantization (nf4)

# My optimized parameters for LoRA training on financial Q&A
min_effective_batch_size = 4  # Reduced for 3.8B model with longer sequences (1024 tokens)
lr = 2e-5  # Slightly lower LR for Phi-3 stability on financial reasoning
max_seq_length = 1024  # Increased for detailed financial Q&A responses
collator_fn = None  # I'm not using a custom collator since I pre-pad in tokenization
packing = False  # I disabled packing since I'm using fixed-length sequences
steps = 20  # My logging and saving frequency
num_train_epochs = 3  # Standard for financial fine-tuning with LoRA
warmup_ratio = 0.1  # Warmup for stable training start

# My SFT configuration for Phi-3-Mini Financial Intelligence
sft_config = SFTConfig(
    # I'm saving my financial model for FinSight 360 project
    output_dir = '/content/drive/MyDrive/phi3-financial/Phi3-Mini-FinancialQA-finetuned',

    # My data processing settings
    packing = packing,
    max_seq_length = max_seq_length,

    # I disabled gradient checkpointing (not needed with 4-bit quantization)
    gradient_checkpointing = False,

    # My training batch and precision settings
    per_device_train_batch_size = min_effective_batch_size,
    auto_find_batch_size = True,  # Let trainer optimize for available VRAM
    bf16 = True,  # Using bf16 for numerical stability in financial calculations

    # My training schedule optimized for financial domain
    num_train_epochs = num_train_epochs,
    learning_rate = lr,
    lr_scheduler_type = "cosine",  # Smooth learning rate decay
    warmup_ratio = warmup_ratio,
    weight_decay = 0.01,  # Regularization for financial terminology
    max_grad_norm = 1.0,  # Gradient clipping for stability

    # My logging and monitoring setup
    report_to = 'wandb',  # Tracking FinSight model training
    run_name = "Phi3-Mini-FinancialQA-LoRA-r8-alpha16",  # Descriptive run name

    # My logging directory
    logging_dir = '/content/drive/MyDrive/phi3-financial/Phi3-Mini-FinancialQA-finetuned/logs',

    # My checkpoint and logging strategy
    logging_strategy = 'steps',
    save_strategy = 'steps',
    logging_steps = steps,  # I log every 20 steps
    save_steps = steps,     # I save checkpoint every 20 steps
    save_total_limit = 2,   # Keep last 2 checkpoints to save space
)

# I'm using subset for faster initial training
train_dataset = final_dataset.select(range(500))  # Using 2k samples for quick iteration

# I create my trainer with the financial Q&A dataset
trainer = SFTTrainer(
    model = model,                    # My Phi-3-Mini with LoRA (rank=8, alpha=16, dropout=0.1)
    train_dataset = train_dataset,    # My 2k financial Q&A samples
    processing_class = tokenizer,     # Phi-3 tokenizer
    data_collator = collator_fn,      # Using default collator
    args = sft_config,               # My financial training configuration
)

# I start the financial fine-tuning process
print("💼 Starting Phi-3-Mini Financial Q&A Fine-tuning...")
print(f"📊 Training on {len(train_dataset)} financial Q&A samples")
print(f"🔧 LoRA Config: rank=8, alpha=16, dropout=0.1")
print(f"💾 4-bit quantization: nf4")
print(f"🎯 Target: FinSight 360 Financial Intelligence System")
print(f"⏱️ Estimated time: ~1.5-2 hours on T4 GPU")
print("-" * 60)

trainer.train()

print("\n" + "="*60)
print("✅ Financial Q&A fine-tuning completed!")
print(f"💾 Model saved to: {sft_config.output_dir}")
print("💼 Your Phi-3 Financial Assistant (FinSight Core) is ready!")
print("="*60)

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


💼 Starting Phi-3-Mini Financial Q&A Fine-tuning...
📊 Training on 500 financial Q&A samples
🔧 LoRA Config: rank=8, alpha=16, dropout=0.1
💾 4-bit quantization: nf4
🎯 Target: FinSight 360 Financial Intelligence System
⏱️ Estimated time: ~1.5-2 hours on T4 GPU
------------------------------------------------------------


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
20,2.770500
40,2.705800
60,2.699800
80,2.469400
100,2.340900


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/

Step,Training Loss
20,2.770500
40,2.705800
60,2.699800
80,2.469400
100,2.340900
120,2.172200
140,2.025700
160,1.850800
180,1.824100
200,1.766400


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/


✅ Financial Q&A fine-tuning completed!
💾 Model saved to: /content/drive/MyDrive/phi3-financial/Phi3-Mini-FinancialQA-finetuned
💼 Your Phi-3 Financial Assistant (FinSight Core) is ready!


In [15]:
# Step 1: I'm saving my trained financial model locally first
print("💾 Saving my trained Phi-3-Mini financial model...")
trainer.save_model('/content/phi3-financial-saved')

# Step 2: I load and merge the LoRA adapter with the base model
from peft import AutoPeftModelForCausalLM

print("🔧 Loading my PEFT model and merging adapter...")
# I load the saved PEFT model (use the same path as Step 1)
peft_model = AutoPeftModelForCausalLM.from_pretrained('/content/phi3-financial-saved')

# I merge and unload the adapter to get a single model
print("⚙️ Merging LoRA weights into base Phi-3 model...")
merged_model = peft_model.merge_and_unload()

# Step 3: I save the merged model with tokenizer
print("💾 Saving my merged financial model...")
merged_model.save_pretrained('/content/phi3-financial-merged')
tokenizer.save_pretrained('/content/phi3-financial-merged')

# Step 4: I upload my model to Hugging Face Hub
from huggingface_hub import HfApi

print("☁️ Uploading my FinSight financial assistant to Hugging Face Hub...")
api = HfApi()
api.upload_folder(
    folder_path='/content/phi3-financial-merged',
    repo_id="sweatSmile/Phi3-Mini-FinSight-FinancialQA",  # My FinSight core model
    repo_type="model",
    commit_message="Upload Phi-3-Mini fine-tuned on 2k FinGPT financial Q&A with LoRA (r=8, alpha=16) - FinSight 360 Core"
)

print("\n" + "="*70)
print("✅ Model upload completed! 🎉")
print("💼 Your Phi-3 FinSight Financial Assistant is now live!")
print("="*70)
print("🔗 Model URL: https://huggingface.co/sweatSmile/Phi3-Mini-FinSight-FinancialQA")
print("\n📋 Model Details:")
print("   - Base: microsoft/Phi-3-mini-4k-instruct (3.8B)")
print("   - Dataset: FinGPT/fingpt-fiqa_qa (2k samples)")
print("   - LoRA: rank=8, alpha=16, dropout=0.1")
print("   - Quantization: 4-bit (nf4)")
print("   - Domain: Financial Q&A for FinSight 360 System")
print("   - Use Case: Investment advice, banking queries, personal finance")
print("="*70)

💾 Saving my trained Phi-3-Mini financial model...
🔧 Loading my PEFT model and merging adapter...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

⚙️ Merging LoRA weights into base Phi-3 model...
💾 Saving my merged financial model...
☁️ Uploading my FinSight financial assistant to Hugging Face Hub...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

  ...0004-of-00004.safetensors:  15%|#4        | 58.7MB /  394MB            

  ...0002-of-00004.safetensors:   0%|          |  554kB / 4.98GB            

  ...0003-of-00004.safetensors:   0%|          | 1.11MB / 4.95GB            

  ...0001-of-00004.safetensors:   0%|          | 16.8MB / 4.96GB            


✅ Model upload completed! 🎉
💼 Your Phi-3 FinSight Financial Assistant is now live!
🔗 Model URL: https://huggingface.co/sweatSmile/Phi3-Mini-FinSight-FinancialQA

📋 Model Details:
   - Base: microsoft/Phi-3-mini-4k-instruct (3.8B)
   - Dataset: FinGPT/fingpt-fiqa_qa (2k samples)
   - LoRA: rank=8, alpha=16, dropout=0.1
   - Quantization: 4-bit (nf4)
   - Domain: Financial Q&A for FinSight 360 System
   - Use Case: Investment advice, banking queries, personal finance
